# Deploy Object Detection Model Use ModelCI

MMDetction is a well-known open source object detection toolbox based on PyTorch. You can refer to <https://arxiv.org/abs/1906.07155> for more details.

By walking through this tutorial, you will be able to:

- Load pretained MMDetction model
- Convert MMDetction model into ONNX format 
- Register and retrieve models by ModelHub

## 1. Prequisities
 
### 1.1 Installation of MMDetction
 
 Firstly you have to install MMDetction according to official instructions : <https://mmdetection.readthedocs.io/en/latest/get_started.html#installation> 

In [1]:
!pip install mmcv-full -f https://download.openmmlab.com/mmcv/dist/cu102/torch1.8.0/index.html
!git clone https://github.com/open-mmlab/mmdetection.git
!cd mmdetection && pip install -q -r requirements/build.txt && pip install -q -v -e .

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Looking in links: https://download.openmmlab.com/mmcv/dist/cu102/torch1.8.0/index.html
     |████████████████████████████████| 16.6 MB 231 kB/s 
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Obtaining file:///home/modelci/NTU/ML-Model-CI/example/notebook/mmdetection
  Attempting uninstall: mmdet
    Found existing installation: mmdet 2.11.0
    Uninstalling mmdet-2.11.0:
      Successfully uninstalled mmdet-2.11.0
  Running setup.py develop for mmdet


### 1.2 Start ModelCI Service
Then we can start our ModelCI service, you should at least set env variables once before starting. You can refer to [last notebook](https://github.com/cap-ntu/ML-Model-CI/blob/master/example/notebook/image_classification_model_deployment.ipynb) for more details.

In [2]:
!modelci service init

2021-04-28 14:52:13.343730: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2021-04-28 14:52:18,839 - ml-modelci Docker Container Manager - INFO - Container name=mongo-27508 stared
2021-04-28 14:52:20,128 - ml-modelci Docker Container Manager - INFO - Container name=cadvisor-36896 started.
2021-04-28 14:52:21,386 - ml-modelci Docker Container Manager - INFO - Container name=dcgm-exporter-23136 started.
2021-04-28 14:52:22,517 - ml-modelci Docker Container Manager - INFO - gpu-metrics-exporter-90458 stared
2021-04-28 14:52:23,028 - modelci backend - INFO - Uvicorn server listening on http://localhost:8000, check full log at /home/modelci/tmp/modelci.log


## 2. Build MMdetection Model
### 2.1 Imports
We should import the following functions:
- preprocess_example_input: for generating tensor and meta info from example image file
- build_model_from_cfg: for building model form config file and checkpoint file

In [3]:
from mmdet.core.export import preprocess_example_input, build_model_from_cfg

### 2.2 Model Config

We should either use a dict or config file for configuration of MMDetection model, to make things simple, we use a config file provided by MMDetection.

Notice: 

- You may need to manually download pretrained model checkpoints from [MMDetection models zoo](https://github.com/open-mmlab/mmdetection/blob/master/docs/model_zoo.md).
- Only a few MMdet models are able to converted into ONNX format, you can refer to [documentation](https://mmdetection.readthedocs.io/en/latest/tutorials/pytorch2onnx.html#list-of-supported-models-exportable-to-onnx) for more detail.

In [4]:
config_file = 'mmdetection/configs/retinanet/retinanet_r50_fpn_1x_coco.py'
checkpoint_file = 'retinanet_r50_fpn_1x_coco_20200130-c2398f9e.pth'

### 2.3 Build Model
Then we can build our MMdetection model based on the configuration above and the checkpoint file we already download.

In [5]:
model = build_model_from_cfg(config_file, checkpoint_file)

Use load_from_local loader


Before conversion, we need to modify forward function to provide the necessary **kwargs parameters such as img_metas.

In order to obtain valid bbox data during the onnx tracing process, we also need to use a tensor generated from image file as model input instead of random tensors.

In [6]:
input_config = {
    'input_shape': (1,3,224,224),
    'input_path': 'mmdetection/demo/demo.jpg',
    'normalize_cfg': {
        'mean': (123.675, 116.28, 103.53),
        'std': (58.395, 57.12, 57.375)
        }
}
one_img, one_meta = preprocess_example_input(input_config)

In [7]:
one_img.shape

torch.Size([1, 3, 224, 224])

In [8]:
from functools import partial
model.forward = partial(model.forward, img_metas=[[one_meta]], return_loss=False)

## 3. Register Model
We can convert the pytorch model above into optimized formats, such as ONNX through modelci

### 3.1 Imports
- modelci.hub.manager: for registering model into ModelHub
- modelci.hub.utils: for generating model save path
- modelci.types.models: for constructing model inputs paramenters

In [9]:
import torch
from pathlib import Path
from modelci.hub.registrar import register_model
from modelci.hub.utils import generate_path_plain
from modelci.types.models import MLModel
from modelci.types.models.common import Engine, Task, Framework, Metric, ModelStatus, IOShape, DataType

### 3.2 Save model

In [10]:
architecture = 'RetinaNet'
framework = Framework.PyTorch
engine = Engine.PYTORCH
version = 1
task = Task.Object_Detection
model_save_path = generate_path_plain(architecture, task, framework, engine, version)

if not Path.is_dir(Path(model_save_path).parent):
    os.makedirs(Path(model_save_path).parent,exist_ok=True)
torch.save(model, model_save_path)

### 3.3 Construct MLModel Instance

Here are some parameters need to be specified before model conversion.
- inputs: The model inputs info
- outputs: The model outputs info
- metric: The evaludation metric data


In [11]:
mlmodel = MLModel(
    weight=Path(model_save_path),
    architecture=architecture,
    dataset='COCO',
    framework=framework,
    engine=engine,
    version=version,
    metric={Metric.mAP: 0.365},
    task=task,
    inputs=[IOShape(name="input", shape=[-1, 3, 204, 204], dtype=DataType.TYPE_FP32)],
    outputs=[
        IOShape(name="BBOX", shape=[-1, 100, 5], dtype=DataType.TYPE_FP32),
        IOShape(name="SCORE", shape=[-1, 100], dtype=DataType.TYPE_FP32)
    ],
    model_status=[ModelStatus.PUBLISHED],
    model_input = [one_img]
)

### 3.3 Register


In [12]:
register_model(mlmodel, convert=True, profile=False)

2021-04-28 17:01:16,961 - converter - WARNING - This model is not supported as torchscript format
2021-04-28 17:01:16,961 - converter - WARNING - This model is not supported as torchscript format
2021-04-28 17:01:16,961 - converter - WARNING - This model is not supported as torchscript format
2021-04-28 17:01:16,961 - converter - WARNING - This model is not supported as torchscript format
2021-04-28 17:01:16,964 - converter - INFO - Use cached model
2021-04-28 17:01:16,964 - converter - INFO - Use cached model
2021-04-28 17:01:16,964 - converter - INFO - Use cached model
2021-04-28 17:01:16,964 - converter - INFO - Use cached model


[MLModel(architecture='RetinaNet', framework=<Framework.PyTorch: 1>, engine=<Engine.PYTORCH: 7>, version=1, dataset='COCO', metric={<Metric.mAP: 1>: 0.365}, task=<Task.Object_Detection: 1>, inputs=[IOShape(shape=[-1, 3, 204, 204], dtype=<DataType.TYPE_FP32: 11>, name='input', format=<ModelInputFormat.FORMAT_NONE: 0>)], outputs=[IOShape(shape=[-1, 100, 5], dtype=<DataType.TYPE_FP32: 11>, name='BBOX', format=<ModelInputFormat.FORMAT_NONE: 0>), IOShape(shape=[-1, 100], dtype=<DataType.TYPE_FP32: 11>, name='SCORE', format=<ModelInputFormat.FORMAT_NONE: 0>)], id=ObjectId('6089245cbf833faa785ce5ad'), parent_model_id=None, weight=Weight(__root__=ObjectId('60892459bf833faa785ce364')), profile_result=None, status=<Status.Unknown: 0>, model_input=None, model_status=[<ModelStatus.PUBLISHED: 0>], creator='modelci', create_time=datetime.datetime(2021, 4, 28, 9, 1, 5, 195276)),
 MLModel(architecture='RetinaNet', framework=<Framework.PyTorch: 1>, engine=<Engine.ONNX: 3>, version=1, dataset='COCO', me

As we could see, MLModelCI support auto conversion of PyTorch models into both torchscript and ONNX format, as a result.

However, this model cannot be transformed into torchscript format, but supportive of ONNX format conversion, there could be serveral factors contributing to model conversion failture such as the model structure and code format.

## 4. Retrieve Model

The following steps will retrieve the model we just registered.

In [13]:
from modelci.hub.manager import retrieve_model
retrieved_models = retrieve_model(
    architecture='RetinaNet',
    framework=Framework.PyTorch,
    version=1
)

In [14]:
retrieved_models

[MLModel(architecture='RetinaNet', framework=<Framework.PyTorch: 1>, engine=<Engine.PYTORCH: 7>, version=1, dataset='COCO', metric={<Metric.mAP: 1>: 0.365}, task=<Task.Object_Detection: 1>, inputs=[IOShape(shape=[-1, 3, 204, 204], dtype=<DataType.TYPE_FP32: 11>, name='input', format=<ModelInputFormat.FORMAT_NONE: 0>)], outputs=[IOShape(shape=[-1, 100, 5], dtype=<DataType.TYPE_FP32: 11>, name='BBOX', format=<ModelInputFormat.FORMAT_NONE: 0>), IOShape(shape=[-1, 100], dtype=<DataType.TYPE_FP32: 11>, name='SCORE', format=<ModelInputFormat.FORMAT_NONE: 0>)], id=ObjectId('6089245cbf833faa785ce5ad'), parent_model_id=None, weight=Weight(__root__=ObjectId('60892459bf833faa785ce364')), profile_result=None, status=<Status.Unknown: 0>, model_input=None, model_status=[<ModelStatus.PUBLISHED: 0>], creator='modelci', create_time=datetime.datetime(2021, 4, 28, 9, 1, 5, 195000, tzinfo=<bson.tz_util.FixedOffset object at 0x7ff4c7957a50>)),
 MLModel(architecture='RetinaNet', framework=<Framework.PyTorch

It's no wonder we get two model objects here cause there is an addition ONNX format model created automatically during previous registering process.

In [15]:
retrieved_models[0].__dict__

{'architecture': 'RetinaNet',
 'framework': <Framework.PyTorch: 1>,
 'engine': <Engine.PYTORCH: 7>,
 'version': 1,
 'dataset': 'COCO',
 'metric': {<Metric.mAP: 1>: 0.365},
 'task': <Task.Object_Detection: 1>,
 'inputs': [IOShape(shape=[-1, 3, 204, 204], dtype=<DataType.TYPE_FP32: 11>, name='input', format=<ModelInputFormat.FORMAT_NONE: 0>)],
 'outputs': [IOShape(shape=[-1, 100, 5], dtype=<DataType.TYPE_FP32: 11>, name='BBOX', format=<ModelInputFormat.FORMAT_NONE: 0>),
  IOShape(shape=[-1, 100], dtype=<DataType.TYPE_FP32: 11>, name='SCORE', format=<ModelInputFormat.FORMAT_NONE: 0>)],
 'id': ObjectId('6089245cbf833faa785ce5ad'),
 'parent_model_id': None,
 'weight': Weight(__root__=ObjectId('60892459bf833faa785ce364')),
 'profile_result': None,
 'status': <Status.Unknown: 0>,
 'model_input': None,
 'model_status': [<ModelStatus.PUBLISHED: 0>],
 'creator': 'modelci',
 'create_time': datetime.datetime(2021, 4, 28, 9, 1, 5, 195000, tzinfo=<bson.tz_util.FixedOffset object at 0x7ff4c7957a50>)}

In [16]:
retrieved_models[1].__dict__

{'architecture': 'RetinaNet',
 'framework': <Framework.PyTorch: 1>,
 'engine': <Engine.ONNX: 3>,
 'version': 1,
 'dataset': 'COCO',
 'metric': {<Metric.mAP: 1>: 0.365},
 'task': <Task.Object_Detection: 1>,
 'inputs': [IOShape(shape=[-1, 3, 204, 204], dtype=<DataType.TYPE_FP32: 11>, name='input', format=<ModelInputFormat.FORMAT_NONE: 0>)],
 'outputs': [IOShape(shape=[-1, 100, 5], dtype=<DataType.TYPE_FP32: 11>, name='BBOX', format=<ModelInputFormat.FORMAT_NONE: 0>),
  IOShape(shape=[-1, 100], dtype=<DataType.TYPE_FP32: 11>, name='SCORE', format=<ModelInputFormat.FORMAT_NONE: 0>)],
 'id': ObjectId('6089245fbf833faa785ce7f5'),
 'parent_model_id': None,
 'weight': Weight(__root__=ObjectId('6089245dbf833faa785ce5ae')),
 'profile_result': None,
 'status': <Status.Unknown: 0>,
 'model_input': None,
 'model_status': [<ModelStatus.CONVERTED: 1>],
 'creator': 'modelci',
 'create_time': datetime.datetime(2021, 4, 28, 9, 1, 5, 195000, tzinfo=<bson.tz_util.FixedOffset object at 0x7ff4c7957a50>)}